# IBM Data Science Capstone
# Week 3 Assignment
# Canadian Postcodes

Import Relevant Libraries

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from urllib.request import urlopen
from bs4 import BeautifulSoup
%matplotlib inline

Scrape Wikipedia Canadian Postcodes page into pandas dataframe

In [10]:
#url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050."   #old URL
html = urlopen(url)
soup = BeautifulSoup(html, 'lxml')

df = (pd.read_html(str(soup.table))[0])

#drop the first row and add column headers
df.drop(axis=0, index=0, inplace=True)
headers = ['Postcode', 'Borough', 'Neighborhood'] 
df.columns = headers

#drop rows with unassigned Boroughs
df = df[df['Borough'] != "Not assigned"].reset_index(drop=True) 
df



,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
...,...,...,...
205,M8Z,Etobicoke,Kingsway Park South West
206,M8Z,Etobicoke,Mimico NW
207,M8Z,Etobicoke,The Queensway West
208,M8Z,Etobicoke,Royal York South West


Remove rows with unassigned Neighborhoods

In [11]:
unassigned = df[df['Neighborhood']=="Not assigned"]

for index in range(len(unassigned)):
    r = unassigned.iloc[index]
    r['Neighborhood']=r['Borough']
    df.iloc[unassigned.index[0]]['Neighborhood'] = r['Borough']

df.head(8)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Etobicoke,Islington Avenue
7,M1B,Scarborough,Rouge


In [12]:
dfg = df.groupby(['Postcode','Borough']).agg({
    'Neighborhood': lambda x: ', '.join(x),
  
}).reset_index()
dfg.drop_duplicates()
dfg

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


In [13]:
url="https://cocl.us/Geospatial_data"
geopost=pd.read_csv(url)

geopost.rename(columns={'Postal Code':'Postcode'}, 
                 inplace=True)
geopost


 

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


Now we merge the geo location columns onto the right hand side of our Postal code dataframe


In [15]:
postalgeo = pd.merge(left=dfg, right=geopost)
postalgeo

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
